<a href="https://colab.research.google.com/github/sonudoo/DSA/blob/master/Machine%20Leaning/ConvolutionUsingTensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
import cv2
from google.colab import drive

In [0]:
drive.mount('/content/drive')

Mounted at /content/drive


In [0]:
X = np.load("/content/drive/My Drive/Colab Notebooks/datasets/SIGNS/X.npy")
Y = np.load("/content/drive/My Drive/Colab Notebooks/datasets/SIGNS/Y.npy")

In [0]:
X = X / 255

In [0]:
m = X.shape[0]
m_train = int(m * 0.8)
m_test = m - m_train

In [0]:
rperm = np.random.permutation(m)
X_train = []
Y_train = []
X_test = []
Y_test = []
for i in range(m_train):
    X_train.append(X[rperm[i]])
    Y_train.append(Y[rperm[i]])

for i in range(m_train, m):
    X_test.append(X[rperm[i]])
    Y_test.append(Y[rperm[i]])

X_train = np.array(X_train, dtype=np.float64)
X_test = np.array(X_test, dtype=np.float64)
Y_train = np.array(Y_train, dtype=np.float64)
Y_test = np.array(Y_test, dtype=np.float64)

In [0]:
X = tf.placeholder(tf.float64, [None, X_train.shape[1], X_train.shape[2], X_train.shape[3]])
Y = tf.placeholder(tf.float64, [None, Y_train.shape[1]])

In [0]:
W1 = tf.Variable(np.random.rand(4, 4, X_train.shape[3], 8), 'W1', dtype=tf.float64)
W2 = tf.Variable(np.random.rand(2, 2, 8, 16), 'W2', dtype=tf.float64)

In [0]:
Z1 = tf.nn.conv2d(X, W1, strides=[1, 1, 1, 1], padding='SAME')
A1 = tf.nn.relu(Z1)
P1 = tf.nn.max_pool(A1, ksize=[1, 8, 8, 1], strides=[1, 8, 8, 1], padding='SAME')
Z2 = tf.nn.conv2d(P1, W2, strides=[1, 1, 1, 1], padding='SAME')
A2 = tf.nn.relu(Z2)
P2 = tf.nn.max_pool(A2, ksize=[1, 4, 4, 1], strides=[1, 4, 4, 1], padding='SAME')
F2 = tf.contrib.layers.flatten(P2)
Z3 = tf.contrib.layers.fully_connected(F2, 10, activation_fn=None)
J = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=Z3, labels=Y))
optimizer = tf.train.AdamOptimizer(0.009).minimize(J)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [0]:
batch_size = 64
n_complete_batches = m_train // batch_size

In [0]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
for i in range(500):
    for j in range(n_complete_batches):
        sess.run([optimizer, J], {X: X_train[j*batch_size: (j+1)*batch_size], Y: Y_train[j*batch_size: (j+1)*batch_size]})
    _, J_opt = sess.run([optimizer, J], {X: X_train[j*n_complete_batches:], Y: Y_train[j*n_complete_batches:]})
    print(i, J_opt)

0 163.80477561108935
1 55.11156275459979
2 12.766942765875028
3 5.241255142854731
4 4.133831950173293
5 3.72876726127107
6 3.484261512535076
7 3.144592056937692
8 3.02171226324614
9 3.0044856926008228
10 2.85002856620125
11 2.6426428372708717
12 2.6202378443689813
13 2.4970759833511558
14 2.4648716910220787
15 2.416964395386517
16 2.427047957141529
17 2.4397129474226045
18 2.423721470025882
19 2.391918844925482
20 2.3672765187930453
21 2.320318592923153
22 2.271290795050888
23 2.237900277132765
24 2.212409641524281
25 2.1879170685981912
26 2.1644402719643914
27 2.141086283330846
28 2.1164424816698126
29 2.0932217751721787
30 2.069890924663357
31 2.04512188013046
32 2.0192791822588982
33 1.991919353213528
34 1.9637513256095132
35 1.9341075164765698
36 1.9043839323135323
37 1.8733149826943296
38 1.8430139722844565
39 1.813170154043177
40 1.7866386337357478
41 1.7634055388471228
42 1.7412405053720277
43 1.7204531904589635
44 1.6928138334372425
45 1.6610208845844172
46 1.619467535220782
47

In [0]:
Y_pred = sess.run([Z3], {X: X_train, Y: Y_train})[0]
Y_pred_test = sess.run([Z3], {X: X_test, Y: Y_test})[0]

In [0]:
print("Training accuracy:", np.sum(np.argmax(Y_train, axis=1) == np.argmax(Y_pred, axis=1)) / m_train)

Training accuracy: 1.0


In [0]:
print("Test accuracy:", np.sum(np.argmax(Y_test, axis=1) == np.argmax(Y_pred_test, axis=1)) / m_test)

Test accuracy: 0.9249394673123487
